In [42]:
from urllib.parse import urlencode
import requests
import time
from datetime import datetime
import json
import pandas as pd

In [2]:
BASE_URL = "http://history.openweathermap.org/data/2.5/history/city?"
API_KEY = open("../wind_farm/api_key.txt").read()

In [33]:
def get_history_weather(params):
    """
    Call OpenWeatherMap historical weather API by coordinates
    Returns JSON file
    """
    query_dict = {"lat": params["lat"],
                  "lon": params["lon"],
                  "type": "hour",
                  "start": params["start"],
                  "end": params["end"],
                  "appid": API_KEY}

    query_str = urlencode(query_dict)

    #print(BASE_URL+query_str)
    api_request = requests.get(BASE_URL+query_str)
    
    return api_request.json()


def get_wind_history(params):
    
    req = get_history_weather(params)
    

In [7]:
params = {"lat": 47.6062,
         "lon": -122.3321,
         "type": "hour",
         "start": int(round(datetime(2020,3,31).timestamp(),0)),
         "end": int(round(datetime(2020,4,1).timestamp(),0))}

In [31]:
req = get_history_weather(params)

http://history.openweathermap.org/data/2.5/history/city?lat=47.6062&lon=-122.3321&type=hour&start=1585638000&end=1585724400&appid=6bce826503c879f393a3f2f7fd8e35b7


In [39]:
req

{'message': 'Count: 25',
 'cod': '200',
 'city_id': 5809844,
 'calctime': 0.00394437,
 'cnt': 25,
 'list': [{'dt': 1585638000,
   'main': {'temp': 277.58,
    'feels_like': 274.49,
    'pressure': 1014,
    'humidity': 86,
    'temp_min': 276.15,
    'temp_max': 278.71},
   'wind': {'speed': 2.1, 'deg': 190},
   'clouds': {'all': 75},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10n'}],
   'rain': {'1h': 0.25}},
  {'dt': 1585641600,
   'main': {'temp': 277.55,
    'feels_like': 274.1,
    'pressure': 1013,
    'humidity': 86,
    'temp_min': 276.15,
    'temp_max': 278.71},
   'wind': {'speed': 2.6, 'deg': 160},
   'clouds': {'all': 90},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10n'}],
   'rain': {'1h': 0.68}},
  {'dt': 1585645200,
   'main': {'temp': 277.44,
    'feels_like': 273.97,
    'pressure': 1014,
    'humidity': 86,
    'temp_min': 275.93,
    'temp_max': 278.71},
   'win

In [72]:
df = pd.DataFrame(data = req["list"])
df.head()

,dt,main,wind,clouds,weather,rain
0,1585638000,"{'temp': 277.58, 'feels_like': 274.49, 'pressu...","{'speed': 2.1, 'deg': 190}",{'all': 75},"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.25}
1,1585641600,"{'temp': 277.55, 'feels_like': 274.1, 'pressur...","{'speed': 2.6, 'deg': 160}",{'all': 90},"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.68}
2,1585645200,"{'temp': 277.44, 'feels_like': 273.97, 'pressu...","{'speed': 2.6, 'deg': 160}",{'all': 75},"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.25}
3,1585648800,"{'temp': 277.35, 'feels_like': 273.17, 'pressu...","{'speed': 3.6, 'deg': 170}",{'all': 75},"[{'id': 803, 'main': 'Clouds', 'description': ...",NaN
4,1585652400,"{'temp': 277.24, 'feels_like': 274.09, 'pressu...","{'speed': 2.1, 'deg': 190}",{'all': 90},"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.3}


In [78]:
df_return = pd.DataFrame(data = df['dt'])

for col in df.columns[1:]:
    if isinstance(df[col][0], dict):
        temp_df = df[col].apply(pd.Series)
    elif isinstance(df[col][0], list):
        temp_df = df[col].apply(pd.Series)[0].apply(pd.Series)
    else:
        temp_df = df[col]
    df_return = df_return.join(temp_df)

In [80]:
def get_wind_history(params):
    """
    Extracts wind speed/direction data from JSON file
    Returns dataframe
    """

    weather_json = get_history_weather(params)

    df = pd.DataFrame(data = weather_json["list"])

    df_return = pd.DataFrame(data = df["dt"])
    
    for col in df.columns[1:]:
        if isinstance(df[col][0], dict):
            temp_df = df[col].apply(pd.Series)
        elif isinstance(df[col][0], list):
            temp_df = df[col].apply(pd.Series)[0].apply(pd.Series)
        else:
            temp_df = df[col]
        df_return = df_return.join(temp_df)
    
    df_return["city_id"] = weather_json["city_id"]
    
    return df_return

In [82]:
skookumchuck = {"lat": 46.741973,
                "lon": -122.639439,
                "type": "hour",
                "start": int(round(datetime(2020,3,31).timestamp(),0)),
                "end": int(round(datetime(2020,4,1).timestamp(),0))}

In [83]:
get_wind_history(skookumchuck)

,dt,temp,feels_like,pressure,humidity,temp_min,temp_max,speed,deg,all,id,main,description,icon,0,1h,city_id
0,1585638000,277.52,274.42,1014.0,86.0,276.15,278.71,2.10,190.0,75,500,Rain,light rain,10n,NaN,0.25,5812944
1,1585641600,277.42,273.25,1014.0,86.0,275.93,278.71,3.60,200.0,90,500,Rain,light rain,10n,NaN,0.34,5812944
2,1585645200,277.34,273.86,1014.0,86.0,275.93,278.71,2.60,160.0,75,803,Clouds,broken clouds,04n,NaN,NaN,5812944
3,1585648800,277.19,272.98,1014.0,86.0,275.93,278.15,3.60,170.0,75,803,Clouds,broken clouds,04n,NaN,NaN,5812944
4,1585652400,277.03,273.68,1014.0,93.0,275.37,278.15,2.60,190.0,90,804,Clouds,overcast clouds,04n,NaN,NaN,5812944
5,1585656000,276.76,273.37,1015.0,93.0,275.15,278.15,2.60,220.0,1,500,Rain,light rain,10n,NaN,0.25,5812944
6,1585659600,276.54,273.12,1014.0,80.0,274.82,278.15,2.10,0.0,90,804,Clouds,overcast clouds,04n,NaN,NaN,5812944
7,1585663200,276.35,272.19,1015.0,93.0,274.82,277.59,3.60,210.0,40,802,Clouds,scattered clouds,03d,NaN,NaN,5812944
8,1585666800,276.55,273.12,1016.0,93.0,275.15,277.59,2.60,230.0,75,803,Clouds,broken clouds,04d,NaN,NaN,5812944
9,1585670400,277.30,273.65,1016.0,80.0,275.93,278.71,2.60,200.0,90,804,Clouds,overcast clouds,04d,NaN,NaN,5812944


In [84]:
wy_wind = {"lat": 41.322294,
           "lon": -110.7984109,
           "type": "hour",
           "start": int(round(datetime(2020,3,31).timestamp(),0)),
           "end": int(round(datetime(2020,4,1).timestamp(),0))}

In [85]:
get_wind_history(wy_wind)

,dt,temp,feels_like,pressure,humidity,temp_min,temp_max,speed,deg,all,id,main,description,icon,city_id
0,1585638000,282.74,276.99,1019.0,57.0,280.37,285.15,5.7,160.0,90,804,Clouds,overcast clouds,04n,5780993
1,1585641600,282.59,275.97,1019.0,53.0,280.37,285.15,6.7,160.0,90,804,Clouds,overcast clouds,04n,5780993
2,1585645200,282.41,275.06,1019.0,53.0,280.15,284.26,7.7,160.0,90,804,Clouds,overcast clouds,04n,5780993
3,1585648800,282.79,276.00,1019.0,57.0,281.15,284.26,7.2,160.0,90,804,Clouds,overcast clouds,04n,5780993
4,1585652400,282.46,276.32,1018.0,57.0,280.37,284.15,6.2,160.0,90,804,Clouds,overcast clouds,04n,5780993
5,1585656000,282.40,275.20,1018.0,57.0,280.15,284.26,7.7,160.0,90,804,Clouds,overcast clouds,04n,5780993
6,1585659600,282.11,277.19,1018.0,61.0,279.82,284.15,4.6,190.0,90,804,Clouds,overcast clouds,04n,5780993
7,1585663200,282.31,276.49,1018.0,66.0,279.26,285.15,6.2,210.0,75,803,Clouds,broken clouds,04d,5780993
8,1585666800,282.69,276.74,1018.0,61.0,280.93,284.15,6.2,160.0,40,802,Clouds,scattered clouds,03d,5780993
9,1585670400,283.64,277.74,1018.0,50.0,280.93,286.15,5.7,150.0,40,802,Clouds,scattered clouds,03d,5780993
